# Explore the different corpora

In [1]:
import sys
sys.path.append("../python/")

In [2]:
import pentoref.IO as IO

In [4]:
# Get the dataframes (can also use a database method instead)
dfwords, dfutts, dfrefs, dfscenes, dfactions = IO.convert_subcorpus_raw_data_to_dataframes("../../TAKECV_PENTOREF")

TAKECV
extracting files in textgrid format
17 Transcription files read.
1828 text files read. 1244 XML files parsed.
Processing:
r5_p2...
r5_p1...
r4_p1...
r4_p2...
r9_p1...
r2_p2...
r2_p1...
r6_p2...
r6_p1...
r3_p1...
r3_p2...
r7_p1...
r8_p2...
r7_p2...
r1_p2...
r1_p1...
r9_p2...
Done.


In [5]:
dfwords

,gameID,position,uttID,word
0,r5_p2_1,1,1,das:
1,r5_p2_1,2,1,hellblaue
2,r5_p2_1,3,1,..
3,r5_p2_1,1,2,Objekt
4,r5_p2_1,2,2,was
5,r5_p2_1,3,2,aus
6,r5_p2_1,4,2,drei
7,r5_p2_1,5,2,Rechtecken
8,r5_p2_1,6,2,besteht
9,r5_p2_1,7,2,befindet


In [5]:
from os import listdir
from os.path import isdir, abspath, join, splitext
from re import match, sub, findall, finditer
from lxml.etree import parse
from lxml.etree import XMLSyntaxError
import lxml.etree
from io import StringIO, BytesIO

In [6]:
parser = lxml.etree.XMLParser(recover=True) #recovers from bad characters.
#tree = lxml.etree.parse(filename, parser)

def get_data_from_xml(xmlpath):
    tiles = dict()
    landmarks = dict()
    scene_descriptions = dict()
    for folder in sorted(listdir(xmlpath)):
        for file in sorted(listdir(join(xmlpath,folder))):
            if not isdir(file):
                name,ext = splitext(file)
                if ext.lower() == '.txt':
                    if 'final-selected' in name:
                        tiles[name[:-15]] = (open(join(xmlpath,folder,file)).read())
                    elif 'landmark' in name:
                        landmarks[name[:-9]] = (open(join(xmlpath,folder,file)).read())
                elif ext.lower() == '.xml' and 's' not in name:
                    try:
                        #if folder == "r7": continue
                        xml = "".join([line for line in open(join(xmlpath,folder,file))])
                        xml = xml.encode("utf-8").decode("utf-8")
                        reader = StringIO(xml)
                        scene_descriptions[name] = xml #lxml.etree.iterparse(reader)
                    except XMLSyntaxError:
                        print "Error parsing file!"
                        print join(xmlpath,folder,file)
                        return None, None, None
    print(str(len(tiles.keys())+len(landmarks.keys()))+' text files read. '+str(len(scene_descriptions))+' XML files parsed.')
    return scene_descriptions

In [7]:
p = get_data_from_xml("/home/dsg-labuser/git/pentoref/PENTOCV_PENTOREF/derived_data/multimodal_data/scene_information")

0 text files read. 38563 XML files parsed.


In [14]:
import sys
import math

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])
print p.values()[0]
convert_size(sys.getsizeof(p.values()[0]))

'32.0 B'

In [5]:
parser = lxml.etree.XMLParser(recover=True) #recovers from bad characters.
p = parse(open("/home/dsg-labuser/git/pentoref/PENTOCV_PENTOREF/derived_data/multimodal_data/scene_information/r8/r8_818543.478261.xml"), parser)

In [6]:
p

In [5]:
count = 0
for line in file:
    count+=1
    if count >226450 and count <226490:
        print line

         <skewness horizontal="left-skewed" vertical="top-skewed"/>

         <nbEdges value="8"/>

      </shape>

      <colour BestResponse="Yellow">

         <distribution Blue="4.09799597348e-43" Brown="0.0229247897728" Gray="6.79334259207e-05" Green="4.70763398076e-11" Orange="0.184595789052" Pink="0.00458168931595" Purple="2.1652803925e-38" Red="0.00020111793191" Yellow="0.787628680454"/>

         <hsvValue H="21.7293526786" S="142.527901786" V="170.111049107"/>

         <rgbValue B="76.9402901786" G="144.027901786" R="169.650111607"/>

      </colour>

   </object>

   <object id="6">

      <position global="center top" x="217" y="156"/>

      <shape BestResponse="T">

         <distribution F="2.35518889062e-18" I="2.78348013814e-25" L="8.45398047286e-19" N="1.8479388816e-18" P="1.01760817162e-18" T="0.762004191676" U="7.41836387378e-19" V="0.237995808324" W="1.13722457289e-16" X="9.79843652367e-30" Y="5.05250197758e-19" Z="1.68931090181e-16"/>

         <orientation valu

In [6]:
dfwords

,endtime,gameID,position,refID,starttime,uttID,word
0,320.583357,r4_1,1,1,320.249396,1,okay
1,323.103442,r4_1,1,1,322.691291,2,ähm
2,323.379596,r4_1,2,1,323.103442,2,also
3,323.635022,r4_1,3,1,323.379596,2,ich
4,323.750636,r4_1,4,1,323.635022,2,hab
5,324.272242,r4_1,5,1,323.828608,2,mir
6,325.772230,r4_1,1,1,325.586710,3,ein
7,326.143270,r4_1,2,1,325.772230,3,Kreuz
8,327.156780,r4_1,3,1,326.143270,3,ausgesucht
9,327.624739,r4_1,4,1,327.377097,3,äh
